In [50]:
from secrets import randbits
from Crypto.Util.number import bytes_to_long, long_to_bytes
import numpy as np
from hashlib import sha512

n = 500
qbits = 32
mbits = 4
q = 2**qbits
F = GF(q)
x = F.gen()

def int_to_F(n):
    return sum(b*x**i for i,b in enumerate(map(int, bin(n)[2:][::-1])))

def F_to_int(f):
    return f.integer_representation()

def gen_key():
    return np.array([b for b in map(int, format(randbits(n), "0500b"))], dtype=object)

def gen_a():
    return np.array([int_to_F(randbits(qbits)) for _ in range(n)], dtype=object)

def gen_noise():
    return int_to_F(randbits(qbits - mbits))

def encrypt_mbits(m, s):
    a = gen_a()
    f = np.vectorize(F_to_int)
    m = int_to_F(m << (qbits - mbits))
    return (f(a), F_to_int(np.dot(a, s) + m + gen_noise()))

def decrypt_mbits(c, s):
    a,b = c
    f = np.vectorize(int_to_F)
    a,b = f(a), int_to_F(b)
    return F_to_int(b - np.dot(a,s)) >> (qbits - mbits)

def encrypt_m(m, s):
    m = bytes_to_long(m)
    c = []
    while m != 0:
        mb = m & 0b1111
        c.append(encrypt_mbits(mb, s))
        m >>= 4
    return c[::-1]

def decrypt_m(c, s):
    m = 0
    for cb in c:
        m <<= 4
        mb = decrypt_mbits(cb, s)
        m += int(mb)
    return long_to_bytes(m)


# https://www.daniellowengrub.com/blog/2024/01/03/fully-homomorphic-encryption
message = b"Original LWE use field GF(prime). TFHE use Mod(2^n). I use GF(2^n)"
key = gen_key()
ciphertext = encrypt_m(message, key)
assert decrypt_m(ciphertext, key) == message

flag = b"grey{I'm_flyin_soon_I'm-_rushing-this-challenge-rn-akeyhashjsdadsdasks}"
keyhash = sha512(long_to_bytes(int(''.join(map(str, key)), 2))).digest()
flag_xored = bytes([a^^b for a,b in zip(flag, keyhash)]).hex()

#print(ciphertext)
print(flag_xored)

e8bd8aa111d1548cb3810aa6bf9084ff47fde9c7e867edea8cdbb7a3577d885e14b551e1210416e304a7ab83411e7cd0db4238a52f06622b933054c2f05c9e41


b'\x08i}\xfc$\xf6\xe0\xd2\xfb\xed\xc8J\xc9\xe4\xa2\xd3\xd8n\xf2\xedS\xcc\xa4\xf8\x9ah\x94\xcc\x06\xa2\xb8\xca)\x1d\x8f.\x06\x8cI\xc7\x1c\xf4\x8fAU\x16u&\xc2+\x94\xcc"\xd2S\xe8\x06\x17\x9b\xf2\x8c\xb3\x0f'

In [2]:
mbs[0], encrupt_mb(mbs[0], key)

NameError: name 'mbs' is not defined

In [60]:
def split_m_into_mb(m):
    m = bytes_to_long(m)
    mbs = []
    while m != 0:
        mbs.append(m & 0b1111)
        m >>= 4
    return mbs[::-1]

def eqns_from_ctbits(cb, mb):
    a,b = cb
    m_bitarr = [*map(int, format(mb, "04b"))]
    m = vector(GF(2), [*map(int, format(mb, "04b"))])
    A = matrix(GF(2), [[*map(int, format(l, "032b")[:mbits])] for l in a]).T
    b = vector(GF(2), [*map(int, format(b, "032b")[:mbits])])
    
    # A * vector(GF(2), [*key]) + m - b = 0
    return A, b - m
    
def matq_solve_right_all(mat, vec, q):
    """
    Solves for all vectors u such that
    `mat * u = vec`, where we are working
    in ZZ/`q`ZZ.
    """
    v = mat.solve_right(vec)
    if is_prime(q):
        return [v + k for k in mat.right_kernel()]
    kernel = modq_kernel(mat, q)
    if kernel.nrows() > 16: raise Exception("Too many solutions, regen challenge")
    unique_v = set(tuple(v + k) for k in FiniteZZsubmodule_iterator([*kernel]))
    return list(vector(Zmod(q), v) for v in unique_v)

mbs = split_m_into_mb(message)
eqns = [eqns_from_ctbits(*x) for x in zip(ciphertext, mbs)]
A = block_matrix([[a] for a,_ in eqns])
m_b = [list(b) for _,b in eqns]
m_b = vector(GF(2), [b for a in m_b for b in a])
for pos_key in matq_solve_right_all(A, m_b, 2):
    xor_stream = sha512(long_to_bytes(int(''.join(map(str, pos_key)), 2))).digest()
    print(bytes([a^^b for a,b in zip(bytes.fromhex(flag_xored), xor_stream)]))

b"grey{I'm_flyin_soon_I'm-_rushing-this-challenge-rn-ajsdadsdasks}"


In [59]:
len("I'm_flyin_soon_I'm-_rushing-this-challenge-rn-ajsdadsdasks")*"X"

'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'

In [74]:
def int_to_F(n):
    return sum(b*x**i for i,b in enumerate(map(int, bin(n)[2:][::-1])))

def F_to_int(f):
    return f.integer_representation()

F_to_int(int_to_F(102802))

102802